# SR <-> (x,y,phi) mapping checks

This notebook validates round-trip mapping between SR coordinates `(s,r,delta)` and cartesian drop parameters `(x,y,phi)`.
The `(x,y,phi) -> (s,r,delta)` mapping uses the nearest SR grid point.


In [1]:
import numpy as np
from scipy.spatial import cKDTree

from fire_model.ca import FireEnv, CAFireModel, FireState
from fire_model.bo_sr import RetardantDropBayesOptSR

np.set_printoptions(precision=4, suppress=True)
rng = np.random.default_rng(0)


In [2]:
# Environment and model.
nx = ny = 120
domain_km = 1.5

fuel = np.ones((nx, ny), dtype=float)
value = np.ones((nx, ny), dtype=float)

wind = np.zeros((nx, ny, 2), dtype=float)
wind[..., 0] = 12.0
wind[..., 1] = -8.0

env = FireEnv(
    grid_size=(nx, ny),
    domain_km=domain_km,
    fuel=fuel,
    value=value,
    wind=wind,
    dt_s=4.0,
    burn_time_s0=500.0,
    retardant_half_life_s=3600.0,
    retardant_k=10.0,
    drop_w_km=0.05,
    drop_h_km=0.2,
    drop_amount=1.0,
    ros_mps=1.0,
    wind_coeff=0.08,
    diag=True,
    avoid_burning_drop=True,
    avoid_drop_p_threshold=0.25,
    ros_future_jitter_frac=0.05,
    wind_coeff_future_jitter_frac=0.05,
)

model = CAFireModel(env, seed=3)


In [6]:
# Initial fire state and SR search grid.
center = (nx // 2, ny // 2)
init_batch = model.init_state_batch(n_sims=20, center=center, radius_km=0.05)
init_state = model.aggregate_mc_to_state(init_batch)

bo = RetardantDropBayesOptSR(
    fire_model=model,
    init_firestate=init_state,
    n_drones=1,
    evolution_time_s=200.0,
    n_sims=20,
    fire_boundary_probability=0.5,
    search_grid_evolution_time_s=200.0,
)

bo.setup_search_grid_sr(
    K=500,
    boundary_field="affected",
    n_r=80,
    smooth_iters=120,
    omega=1.0,
)

print("SR grid shape:", bo.sr_grid.shape)
print("Valid points:", int(np.sum(bo.sr_valid_mask)))


No drone_params
SR grid shape: (500, 80, 2)
Valid points: 39530


In [7]:
# Helpers for SR <-> XY+phi mapping.
K, R = bo.sr_grid.shape[:2]
coords = bo.sr_valid_indices.astype(int)
xy_points = bo.sr_grid[coords[:, 0], coords[:, 1]]
tree = cKDTree(xy_points)

def angle_diff(a, b):
    return (a - b + np.pi) % (2.0 * np.pi) - np.pi

def sr_to_xy_phi(bo, s, r, delta):
    xy, phi_s = bo._sr_lookup(s, r)
    phi_long = phi_s + float(delta)
    phi = bo._phi_from_long_axis_angle(phi_long)
    return np.array([xy[0], xy[1], phi], dtype=float)

def xy_phi_to_sr(bo, xy, phi, tree, coords):
    dist, idx = tree.query(xy, k=1)
    i, j = coords[int(idx)]
    s = i / float(K - 1)
    r = j / float(R - 1)
    phi_s = float(bo.sr_phi_grid[i, j])
    phi_long = bo._wrap_angle(0.5 * np.pi - float(phi))
    delta = bo._wrap_angle(phi_long - phi_s)
    return np.array([s, r, delta], dtype=float), float(dist)


In [8]:
# Round-trip tests: SR -> XY -> SR, and XY -> SR -> XY.
n_samples = min(200, coords.shape[0])
sel = rng.choice(coords.shape[0], size=n_samples, replace=False)

s_err = []
r_err = []
delta_err = []
xy_err = []
phi_err = []
dist_err = []

for idx in sel:
    i, j = coords[idx]
    s = i / float(K - 1)
    r = j / float(R - 1)
    delta = rng.random() * 2.0 * np.pi

    xy_phi = sr_to_xy_phi(bo, s, r, delta)
    sr_back, dist = xy_phi_to_sr(bo, xy_phi[:2], xy_phi[2], tree, coords)
    xy_back = sr_to_xy_phi(bo, sr_back[0], sr_back[1], sr_back[2])

    s_err.append(abs(sr_back[0] - s))
    r_err.append(abs(sr_back[1] - r))
    delta_err.append(abs(angle_diff(sr_back[2], delta)))
    xy_err.append(np.linalg.norm(xy_back[:2] - xy_phi[:2]))
    phi_err.append(abs(angle_diff(xy_back[2], xy_phi[2])))
    dist_err.append(dist)

print("s error (max, mean):", float(np.max(s_err)), float(np.mean(s_err)))
print("r error (max, mean):", float(np.max(r_err)), float(np.mean(r_err)))
print("delta error (max, mean):", float(np.max(delta_err)), float(np.mean(delta_err)))
print("xy error (max, mean):", float(np.max(xy_err)), float(np.mean(xy_err)))
print("phi error (max, mean):", float(np.max(phi_err)), float(np.mean(phi_err)))
print("nearest-grid dist (max, mean):", float(np.max(dist_err)), float(np.mean(dist_err)))

assert np.max(s_err) < 1e-6
assert np.max(r_err) < 1e-6
assert np.max(delta_err) < 1e-6
assert np.max(xy_err) < 1e-6
assert np.max(phi_err) < 1e-6
assert np.max(dist_err) < 1e-6


s error (max, mean): 0.0 0.0
r error (max, mean): 0.0 0.0
delta error (max, mean): 8.881784197001252e-16 1.7541523789077474e-16
xy error (max, mean): 0.0 0.0
phi error (max, mean): 8.881784197001252e-16 5.551115123125783e-17
nearest-grid dist (max, mean): 0.0 0.0
